In [34]:
# Test 1

input = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

In [28]:
# Test 2

input = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

In [36]:
# Real

input = """REDACTED"""

In [46]:
# Part 2 input hacks

# Derived from a graphviz drawing
# For each, only have the nth broadcaster start and then remove the final link and then rerun the part 2 cell

input1 = """REDACTED"""

input2 = """REDACTED"""

input3 = """REDACTED"""

input4 = """REDACTED"""

In [37]:
from collections import deque

# map name to (type, output list)
nodes = {}
# map names to True if On
flip_node_states = {}
# map names to (map names to High/Low)
con_node_states = {}

for line in input.splitlines():
    type = "" # for boring
    name = ""
    if line[0] not in ['%', '&']:
        name = line.split(" -> ")[0]
        type = "B" # for boring
        outputs = [x.strip() for x in line.split(" -> ")[1].split(',')]
    elif line[0] == '%':
        name = line.split(" -> ")[0][1:]
        type = "%"
        outputs = [x.strip() for x in line.split(" -> ")[1].split(',')]
        flip_node_states[name] = False
    else:
        name = line.split(" -> ")[0][1:]
        type = "&"
        outputs = [x.strip() for x in line.split(" -> ")[1].split(',')]
    nodes[name] = (type, outputs)

for k, v in nodes.items():
    connections = {}
    if v[0] == '&':
        for k2, v2 in nodes.items():
            if k in v2[1]:
                connections[k2] = "L"
        con_node_states[k] = connections


pulses = deque()
count_low = 0
count_high = 0

for i in range(1000):
    pulses.append(("broadcaster","L", "button"))

    while len(pulses) > 0:
        current = pulses.popleft()
        if current[1] == "L":
            count_low += 1
        else:
            count_high += 1
        # Special case on "broadcaster"
        if current[0] == "broadcaster":
            for dest in nodes["broadcaster"][1]:
                pulses.append((dest, current[1], "broadcaster"))
            continue
        else:
            if current[0] not in nodes:
                continue
            type = nodes[current[0]][0]
            if type == "%":
                if current[1] == "H":
                    # High to flip-flop does nothing
                    continue
                else:
                    # flip state
                    flip_node_states[current[0]] = not flip_node_states[current[0]]
                    out = "H" if flip_node_states[current[0]] else "L"
                    for dest in nodes[current[0]][1]:
                        pulses.append((dest, out, current[0]))
                    continue
            elif type == "&":
                # update state
                con_node_states[current[0]][current[2]] = current[1]
                out = "L"
                for state in con_node_states[current[0]].values():
                    if state == "L":
                        out = "H"
                for dest in nodes[current[0]][1]:
                    pulses.append((dest, out, current[0]))
                continue

print("Part 1: ", count_high * count_low)


            
            
        

Part 1:  730797576


In [58]:
# PART 2
from math import lcm

# We do assume here that once we hit a high output we immediately loop exactly which appears to be true but definitely is not supported textually

inputs = [input1, input2, input3, input4]
loops = []

for inp in inputs:

    # map name to (type, output list)
    nodes = {}
    # map names to True if On
    flip_node_states = {}
    # map names to (map names to High/Low)
    con_node_states = {}

    for line in inp.splitlines():
        type = "" # for boring
        name = ""
        if line[0] not in ['%', '&']:
            name = line.split(" -> ")[0]
            type = "B" # for boring
            outputs = [x.strip() for x in line.split(" -> ")[1].split(',')]
        elif line[0] == '%':
            name = line.split(" -> ")[0][1:]
            type = "%"
            outputs = [x.strip() for x in line.split(" -> ")[1].split(',')]
            flip_node_states[name] = False
        else:
            name = line.split(" -> ")[0][1:]
            type = "&"
            outputs = [x.strip() for x in line.split(" -> ")[1].split(',')]
        nodes[name] = (type, outputs)

    for k, v in nodes.items():
        connections = {}
        if v[0] == '&':
            for k2, v2 in nodes.items():
                if k in v2[1]:
                    connections[k2] = "L"
            con_node_states[k] = connections

    pulses = deque()
    loop_length = 0
    stay = True

    while stay:
        pulses.append(("broadcaster","L", "button"))
        loop_length += 1
        # if loop_length % 10000 == 0:
        #     print(loop_length)
        while len(pulses) > 0:
            current = pulses.popleft()
            if current[0] == "out" and current[1] == "H":
                print(loop_length)   
                stay = False     
            # Special case on "broadcaster"
            if current[0] == "broadcaster":
                for dest in nodes["broadcaster"][1]:
                    pulses.append((dest, current[1], "broadcaster"))
                continue
            else:
                if current[0] not in nodes:
                    continue
                type = nodes[current[0]][0]
                if type == "%":
                    if current[1] == "H":
                        # High to flip-flop does nothing
                        continue
                    else:
                        # flip state
                        flip_node_states[current[0]] = not flip_node_states[current[0]]
                        out = "H" if flip_node_states[current[0]] else "L"
                        for dest in nodes[current[0]][1]:
                            pulses.append((dest, out, current[0]))
                        continue
                elif type == "&":
                    # update state
                    con_node_states[current[0]][current[2]] = current[1]
                    out = "L"
                    for state in con_node_states[current[0]].values():
                        if state == "L":
                            out = "H"
                    for dest in nodes[current[0]][1]:
                        pulses.append((dest, out, current[0]))
                    continue
    loops.append(loop_length)

print ("Part 2:" , lcm(*loops))      

4057
3793
3947
3733
Part 2: 226732077152351


Part 2: 226732077152351


In [ ]:
# Loop lengths:

# 1 : 4057
# 2 : 3793
# 3 : 3947
# 4 : 3733